In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
english_sentences=pd.read_csv('ef.csv')

In [ ]:
french_sentences=pd.read_csv('ef.csv')

In [ ]:
tokenizer_en = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer_en.fit_on_texts(english_sentences)
eng_seq = tokenizer_en.texts_to_sequences(english_sentences)
eng_seq = pad_sequences(eng_seq, padding='post')
tokenizer_fr = tf.keras.preprocessing.text.Tokenizer(filters='')
tokenizer_fr.fit_on_texts(french_sentences)
fr_seq = tokenizer_fr.texts_to_sequences(french_sentences)
fr_seq = pad_sequences(fr_seq, padding='post')

In [ ]:
vocab_size_en = len(tokenizer_en.word_index) + 1
vocab_size_fr = len(tokenizer_fr.word_index) + 1


In [ ]:
input_shape = eng_seq.shape[1]
embedding_dim = 256
hidden_units = 512


In [ ]:
encoder_inputs = Input(shape=(input_shape,))
encoder_embedding = Masking(mask_value=0)(encoder_inputs)
encoder_lstm = keras.layers.LSTM(hidden_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(input_shape,))
decoder_embedding = Masking(mask_value=0)(decoder_inputs)
decoder_lstm = keras.layers.LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size_fr, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit([eng_seq, fr_seq[:, :-1]], np.expand_dims(fr_seq[:, 1:], -1), batch_size=64, epochs=30, validation_split=0.2)


In [ ]:
def translate_sentence(sentence):
    sentence_seq = tokenizer_en.texts_to_sequences([sentence])
    sentence_seq = pad_sequences(sentence_seq, padding='post', maxlen=input_shape)
    translated_seq = np.argmax(model.predict([sentence_seq, np.zeros((len(sentence_seq), input_shape))]), axis=-1)
    translated_sentence = ' '.join([tokenizer_fr.index_word[i] for i in translated_seq[0]])
    return translated_sentence

In [ ]:
english_sentence = "Hello, how are you?"
translated_sentence = translate_sentence(english_sentence)
print(translated_sentence)